In [2]:
import numpy as np
import pandas as pd

datapath = './data/Hostel2.csv'#iso-8859-1 에서 utf-8로 인코딩
data1 = pd.read_csv(datapath, sep=',', encoding='utf-8')

data1.shape

(342, 16)

In [3]:
data1.head()

,Unnamed: 0,hostel.name,City,price.from,Distance,summary.score,rating.band,atmosphere,cleanliness,facilities,location.y,security,staff,valueformoney,lon,lat
0,1,"""Bike & Bed"" CharinCo Hostel",Osaka,3300,2.9km from city centre,9.2,Superb,8.9,9.4,9.3,8.9,9.0,9.4,9.4,135.513767,34.682678
1,2,& And Hostel,Fukuoka-City,2600,0.7km from city centre,9.5,Superb,9.4,9.7,9.5,9.7,9.2,9.7,9.5,NaN,NaN
2,3,&And Hostel Akihabara,Tokyo,3600,7.8km from city centre,8.7,Fabulous,8.0,7.0,9.0,8.0,10.0,10.0,9.0,139.777472,35.697447
3,4,&And Hostel Ueno,Tokyo,2600,8.7km from city centre,7.4,Very Good,8.0,7.5,7.5,7.5,7.0,8.0,6.5,139.783667,35.712716
4,5,&And Hostel-Asakusa North-,Tokyo,1500,10.5km from city centre,9.4,Superb,9.5,9.5,9.0,9.0,9.5,10.0,9.5,139.798371,35.727898


In [4]:
filtered_data = data1.dropna()#결측값있는 행 제거
del(filtered_data['Unnamed: 0'])
del(filtered_data['hostel.name'])#필요없는 열 제거
filtered_data['Distance']=filtered_data['Distance'].apply(lambda x: x.replace('km from city centre',''))
filtered_data.head()

/home/u1019/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,City,price.from,Distance,summary.score,rating.band,atmosphere,cleanliness,facilities,location.y,security,staff,valueformoney,lon,lat
0,Osaka,3300,2.9,9.2,Superb,8.9,9.4,9.3,8.9,9.0,9.4,9.4,135.513767,34.682678
2,Tokyo,3600,7.8,8.7,Fabulous,8.0,7.0,9.0,8.0,10.0,10.0,9.0,139.777472,35.697447
3,Tokyo,2600,8.7,7.4,Very Good,8.0,7.5,7.5,7.5,7.0,8.0,6.5,139.783667,35.712716
4,Tokyo,1500,10.5,9.4,Superb,9.5,9.5,9.0,9.0,9.5,10.0,9.5,139.798371,35.727898
5,Tokyo,2100,9.4,7.0,Very Good,5.5,8.0,6.0,6.0,8.5,8.5,6.5,139.786950,35.724384


In [5]:
filtered_data.shape

(298, 14)

In [6]:
from sklearn import preprocessing

le_city = preprocessing.LabelEncoder()
le_city.fit(filtered_data['City'])
le_rating = preprocessing.LabelEncoder()
le_rating.fit(filtered_data['rating.band'])


print(le_city.classes_)
print(le_city.transform(['Fukuoka-City', 'Hiroshima', 'Kyoto', 'Osaka', 'Tokyo']))
print(le_city.inverse_transform([0, 1, 2, 3, 4]))
print(le_rating.classes_)
print(le_rating.transform(['Fabulous', 'Good', 'Rating', 'Superb', 'Very Good']))
print(le_rating.inverse_transform([0, 1, 2, 3, 4]))

['Fukuoka-City' 'Hiroshima' 'Kyoto' 'Osaka' 'Tokyo']
[0 1 2 3 4]
['Fukuoka-City' 'Hiroshima' 'Kyoto' 'Osaka' 'Tokyo']
['Fabulous' 'Good' 'Rating' 'Superb' 'Very Good']
[0 1 2 3 4]
['Fabulous' 'Good' 'Rating' 'Superb' 'Very Good']


In [20]:
dicted_data = filtered_data.copy()
dicted_data['City']=le_city.transform(filtered_data['City'])
dicted_data['rating.band']=le_rating.transform(filtered_data['rating.band'])

dicted_data.head()

,City,price.from,Distance,summary.score,rating.band,atmosphere,cleanliness,facilities,location.y,security,staff,valueformoney,lon,lat
0,3,3300,2.9,9.2,3,8.9,9.4,9.3,8.9,9.0,9.4,9.4,135.513767,34.682678
2,4,3600,7.8,8.7,0,8.0,7.0,9.0,8.0,10.0,10.0,9.0,139.777472,35.697447
3,4,2600,8.7,7.4,4,8.0,7.5,7.5,7.5,7.0,8.0,6.5,139.783667,35.712716
4,4,1500,10.5,9.4,3,9.5,9.5,9.0,9.0,9.5,10.0,9.5,139.798371,35.727898
5,4,2100,9.4,7.0,4,5.5,8.0,6.0,6.0,8.5,8.5,6.5,139.786950,35.724384


In [145]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

features = ['staff','security','cleanliness','Distance','atmosphere','facilities','valueformoney','rating.band','City']

kf = KFold(n_splits=10, shuffle=True)

accrs = []
fold_idx = 1
for train_idx, test_idx in kf.split(dicted_data):
    print('Fold {} train size = {}, test size = {}'.format(fold_idx, len(train_idx),len(test_idx)))
    train_d, test_d = dicted_data.iloc[train_idx], dicted_data.iloc[test_idx]
    
    train_y = train_d['price.from']
    train_x= train_d[features]
    
    test_y = test_d['price.from']
    test_x = test_d[features]
    
    model = LinearRegression()
    model.fit(train_x, train_y)
    
    mean_accr = model.score(test_x, test_y)
    print('MAE ={}'.format(mean_accr))
    accrs.append(mean_accr)
    
    
    fold_idx += 1
    
print(np.average(accrs))

Fold 1 train size = 268, test size = 30
MAE =0.17774830163020205
Fold 2 train size = 268, test size = 30
MAE =0.052316581273365796
Fold 3 train size = 268, test size = 30
MAE =-0.27686794671938353
Fold 4 train size = 268, test size = 30
MAE =-0.0025740864058552138
Fold 5 train size = 268, test size = 30
MAE =0.029185721941725973
Fold 6 train size = 268, test size = 30
MAE =0.06882606078030928
Fold 7 train size = 268, test size = 30
MAE =-0.16591050084772951
Fold 8 train size = 268, test size = 30
MAE =0.0919179027774133
Fold 9 train size = 269, test size = 29
MAE =0.14679953477364016
Fold 10 train size = 269, test size = 29
MAE =-0.4168915024815496
-0.02954499332778613
